In [30]:
import pandas as pd
import numpy as np

In [31]:
# Load your CSV
df = pd.read_csv("results.txt")  # Update path if needed

df = df.drop(columns=['NOTHING'], axis=1)  # Drop the index column if it exists

# Keep the best (fastest time) row for each (type, size, cores, blockSize)
best_df = df.loc[df.groupby(['type', 'size', 'cores', 'blockSize'])['time'].idxmin()]

# Sort for readability
best_df = best_df.sort_values(by=['type', 'cores', 'blockSize', 'size'])

# Save to a new file
best_df.to_csv("best_results.csv", index=False)

print("Best results saved to best_results.csv")


Best results saved to best_results.csv


In [32]:
sequential_df = df[df["type"] == "Sequential"]
best_seq = sequential_df.loc[sequential_df.groupby("size")["time"].idxmin()][["size", "time"]]
best_seq = best_seq.rename(columns={"time": "seq_time"})


print(best_seq)

     size      seq_time
405  1024    113.591071
406  2048    920.639072
407  3072   3113.594459
408  4096   7472.435349
409  5120  14639.751450
410  6144  25542.947952
22   7168  41376.450127
412  8192  62302.208832


In [33]:
# Merge best results with best sequential times
best_df = best_df.merge(best_seq, on="size", how="left")

# Calculate speedup
best_df["speedup"] = best_df["seq_time"] / best_df["time"]

# Sort and save
best_df = best_df.sort_values(by=['type', 'cores', 'blockSize', 'size'])

print(best_df.head())

best_df.to_csv("best_results.csv", index=False)

       type  size  cores  blockSize          time      energy       L1_DCM  \
0  Parallel  1024      1         64    117.054505    2.037050    187721621   
1  Parallel  2048      1         64    994.937664   15.554372   1516485456   
2  Parallel  3072      1         64   3377.861558   50.610663   5092297630   
3  Parallel  4096      1         64   8093.431016  124.158359  12162142311   
4  Parallel  5120      1         64  15685.688488  239.448234  23556327797   

      L2_DCM        FP_INS       TOT_INS      seq_time   speedup  
0    3308548    2147483650    1431421562    113.591071  0.970412  
1   26570333   17179869186   11451153833    920.639072  0.925323  
2   90361219   57982058498   38647616841   3113.594459  0.921765  
3  215039632  137438953474   91609145830   7472.435349  0.923272  
4  403196416  268435456002  178924094930  14639.751450  0.933319  


In [34]:
# Time in seconds
best_df["time_s"] = best_df["time"] / 1000.0

# EDP: Energy * Time
best_df["edp"] = best_df["energy"] * best_df["time_s"]

# Power Efficiency (W): Energy / Time
best_df["power_efficiency"] = best_df["time_s"] / best_df["energy"] 

# Instantaneous Power (W): Energy / Time in seconds
best_df["instantaneous_power"] = best_df["energy"] / best_df["time_s"]

# GFLOPS
best_df["gflops"] = (2 * best_df["size"] ** 3) / (best_df["time_s"] * 1e9)
best_df["joules_per_gflop"] = best_df["energy"] / best_df["gflops"]

# cache miss rate
best_df["cache_miss_rate"] = best_df["L1_DCM"] / best_df["TOT_INS"]

# Save final results with EDP and power efficiency
best_df.to_csv("best_results.csv", index=False)